In [11]:
from markdownify import markdownify as md
from datasets import load_dataset
import re

In [4]:
dataset = load_dataset("asavanovich/sft_dataset")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/alex/.cache/huggingface/datasets/asavanovich___parquet/asavanovich--sft_dataset-5ea0b5130193e881/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
code_examples = []
for r in dataset['train']:
    if '```' in r['question']:
        code_examples.append(r['question'])
    if '```' in r['response_j']:
        code_examples.append(r['response_j'])

In [9]:
len(code_examples)

1107

In [28]:
""+code_examples[0]

'I\'ve written at great length about [secure logins and "remember me" checkboxes](https://paragonie.com/blog/2015/04/secure-authentication-php-with-long-term-persistence#title.2.1). The accepted answer isn\'t wrong, but I\'d argue that it\'s a bit more complicated in one respect than it needs to be, and neglects an area where a little bit more complexity is needed.\n\n> \n> generate and store a nonce on the server side, hash that with the username and other info (e.g. client ip, computername, timestamp, similar stuff), and send that in the cookie. The nonce should be stored in the database, together with expiry date, and both checked when the cookie comes back.\n> \n> \n> \n\nSo an implementation that doesn\'t expose any information to the client might look like...\n\n```\n// Storage:\nsetcookie(\n    \'rememberme\',\n    hash_hmac(\'sha256\', $username . $_SERVER[\'REMOTE_ADDR\'], $storedNonce),\n    time() + 8640000 // 100 days, for example\n);\n// Validation:\n$valid = hash_equals(\

In [122]:
R_CODE_BLOCK = re.compile('^(([ \t]*`{3,4})([^\n]*)([\s\S]+?)(^[ \t]*`{3,4}))', re.MULTILINE)
code_ranges = []
for m in re.finditer(R_CODE_BLOCK, code_examples[0]['response_j']):
    code_ranges.append((m.start(), m.end()))
R_CODE = re.compile('(?<=`)[^`\r\n]+(?=`)')
for m in re.finditer(R_CODE, code_examples[0]['response_j']):
    code_ranges.append((m.start(), m.end()))

In [123]:
code_ranges

[(763, 1080),
 (1783, 1797),
 (2112, 2119),
 (2120, 2125),
 (2126, 2151),
 (2231, 2251),
 (2252, 2285),
 (2286, 2296),
 (2362, 2369),
 (2370, 2375),
 (2376, 2386),
 (2428, 2435),
 (2497, 2560),
 (2585, 2589),
 (3210, 3220)]

In [125]:
number = 764
in_range = any(start <= number <= end for start, end in code_ranges)
in_range

True

In [126]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas

In [160]:
aug = nac.KeyboardAug(
#     aug_char_max=1, 
                      aug_word_p=100,
                      stopwords_regex='[^\w]',
                     #include_special_char=False, include_numeric=False, include_upper_case=False,
                     #min_char=1
                     )

In [161]:
text = code_examples[0]['response_j']
prev_end = 0
for r in code_ranges:
    substr = text[prev_end:r[0]]
    prev_end = r[1]
    print(substr)
    print("="*20)
    print(aug.augment(substr)[0])
    print("="*40)
    break

I've written at great length about [secure logins and "remember me" checkboxes](https://paragonie.com/blog/2015/04/secure-authentication-php-with-long-term-persistence#title.2.1). The accepted answer isn't wrong, but I'd argue that it's a bit more complicated in one respect than it needs to be, and neglects an area where a little bit more complexity is needed.

> 
> generate and store a nonce on the server side, hash that with the username and other info (e.g. client ip, computername, timestamp, similar stuff), and send that in the cookie. The nonce should be stored in the database, together with expiry date, and both checked when the cookie comes back.
> 
> 
> 

So an implementation that doesn't expose any information to the client might look like...


I ' ve written at great length about [secure logins and " remember me " checkboxes] (https: / / paragonie. com / blog / 2015 / 04 / secure - authentication - php - with - long - term - persistence # title. 2. 1 ). The accepted answer is

In [170]:
import augmenty
import spacy
nlp = spacy.load("en_core_web_md")
keystroke_error_augmenter = augmenty.load("keystroke_error_v1", level=0.1)

In [171]:
text = code_examples[0]['response_j']


texts = []
prev_end = 0
for r in code_ranges:
    substr = text[prev_end:r[0]]
    prev_end = r[1]
    texts.append(substr)
    
augmented_texts = augmenty.texts(texts, augmenter=keystroke_error_augmenter, nlp=nlp)
    
for text, augmented_text in zip(texts, augmented_texts):
    print(text)
    print("="*20)
    print(augmented_text)
    print("="*40)
    break

I've written at great length about [secure logins and "remember me" checkboxes](https://paragonie.com/blog/2015/04/secure-authentication-php-with-long-term-persistence#title.2.1). The accepted answer isn't wrong, but I'd argue that it's a bit more complicated in one respect than it needs to be, and neglects an area where a little bit more complexity is needed.

> 
> generate and store a nonce on the server side, hash that with the username and other info (e.g. client ip, computername, timestamp, similar stuff), and send that in the cookie. The nonce should be stored in the database, together with expiry date, and both checked when the cookie comes back.
> 
> 
> 

So an implementation that doesn't expose any information to the client might look like...


I've wrirten at ggrat lengty about [sec7rf l8gkns and "remember me" checkboxes](https://paragonie.cpmlblog/2015l04/secure-authenticwtion-php-with-lomg-twrm0persistence#title.2.1(k The acxepted anwwer isn't wrong, but I'd argue thay ih's